In [1]:
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.multioutput import MultiOutputRegressor
import warnings
warnings.filterwarnings('ignore')

### Helper Functions

In [2]:
def loadData(file):
    data = pd.read_csv(file)
    print('Raw shape: ',data.shape)
    data['Date'] = pd.to_datetime(data.Date)
    print('Days: ',len(set(data.Date)))
    return data

In [3]:
def getTimeSeries(df):
    table = pd.pivot_table(df, values='vehicle_count', index=['Date','Hour'],
                    columns=['DOLocationID'], aggfunc=np.sum, fill_value=0)
    return table

In [4]:
def zscoreNormalizeSpatial(matrix):
    m = matrix.copy()
    for i in range(m.shape[0]):
        m[i, :] = (m[i, :] - m[i, :].mean()) / (m[i, :].std()+1e-10)
        
    return m

In [5]:
def standardize(matrix):
    m = matrix.copy()
    scaler = StandardScaler()
    scaler.fit(m)
    t = scaler.transform(m)
    return scaler, t
def inverse_standardize(matrix, scaler):
    t = matrix.copy()
    return scaler.inverse_transform(t)
def getPCAFeatures(matrix, n=10):
    pca = PCA(n_components=n)
    pca.fit(matrix)
    reducedMatrixPCA = pca.transform(matrix)
    reducedMatrixPCA.shape

    reducedDict = {str(i+1):reducedMatrixPCA[:,i] for i in range(reducedMatrixPCA.shape[1])}
    reducedDf = pd.DataFrame(reducedDict)
    #reducedDf.index = index
    return pca,reducedDf

In [6]:
def inverse_standardize(matrix, scaler):
    t = matrix.copy()
    return scaler.inverse_transform(t)

In [7]:
def getPCAFeatures(matrix, n=10):
    pca = PCA(n_components=n)
    pca.fit(matrix)
    reducedMatrixPCA = pca.transform(matrix)
    reducedMatrixPCA.shape

    reducedDict = {str(i+1):reducedMatrixPCA[:,i] for i in range(reducedMatrixPCA.shape[1])}
    reducedDf = pd.DataFrame(reducedDict)
    #reducedDf.index = index
    return pca,reducedDf

In [8]:
def PCA_test(matrix, pca):

    reducedMatrixPCA = pca.transform(matrix)

    reducedDict = {str(i+1):reducedMatrixPCA[:,i] for i in range(reducedMatrixPCA.shape[1])}
    reducedDf = pd.DataFrame(reducedDict)
    #reducedDf.index = index
    return reducedDf

In [9]:
def inverse_pca(matrix,pca):
    m = matrix.copy()
    return pca.inverse_transform(m)

In [10]:
def addLag(dataset, maxlag, lagColumns):
    dataset_list = [dataset]

    for l in range(1, maxlag+1):
        df = dataset.shift(l)
        df = df[lagColumns]
        df.columns = [c+'_lag_'+str(l) for c in df.columns]
        dataset_list.append(df)

    dataset = pd.concat(dataset_list, axis=1).dropna()
    return dataset

In [11]:
def get_rmse(matrix1, matrix2):
    sumSquareError = np.mean(np.power(matrix1 - matrix2,2))
    rmse = np.power(sumSquareError,0.5)
    return rmse

In [12]:
def pca_performance(trainmatrix,testmatrix, components):
    rmseList = []
    r2List = []
    for n in components:
        scaler, s_train_matrix = standardize(trainmatrix)
        s_test_matrix = scaler.transform(testmatrix)

        pca,pcaTrain = getPCAFeatures(s_train_matrix,n=n)
        pcaTest = PCA_test(s_test_matrix, pca)
        
        network_prediction = inverse_pca(pcaTest,pca)
        network_prediction = inverse_standardize(network_prediction, scaler)

        r2Score = r2_score(testmatrix, network_prediction, multioutput='variance_weighted')
                
        r2List.append(r2Score)
    
    return r2List

In [13]:
def nonlinearperformance(trainmatrix,testmatrix,components, maxlag=12):
    r2List = []
    for n in components:
        print(n)
        scaler, s_train_matrix = standardize(trainmatrix)
        s_test_matrix = scaler.transform(testmatrix)

        pca,pcaTrain = getPCAFeatures(s_train_matrix,n=n)
        pcaTest = PCA_test(s_test_matrix, pca)

#         maxlag = 12
        DateColumns = ['Date', 'Hour']
        lagColumns = [c for c in pcaTrain.columns if c not in DateColumns]

        dataset_train = addLag(pcaTrain, maxlag)

        dataset_test = addLag(pcaTest, maxlag)

        X_train = dataset_train.drop(lagColumns , axis = 1)
        X_test = dataset_test.drop(lagColumns , axis = 1)
        y_train = dataset_train[lagColumns]
        y_test = dataset_test[lagColumns]
#         print(X_train.shape)
#         print(X_test.shape)
#         print(y_train.shape)
#         print(y_test.shape)

        rf2 = RandomForestRegressor(random_state = 0, n_estimators=200, 
                                   min_samples_split=10,
                                   min_samples_leaf= 3, 
                                   max_features= 'sqrt',
                                   max_depth= 30, 
                                   bootstrap= True)

        rf2.fit(X_train,y_train)

        pca_prediction = rf2.predict(X_test)

        network_prediction = inverse_pca(pca_prediction,pca)

        network_prediction = inverse_standardize(network_prediction, scaler)

        r2Score = r2_score(testmatrix[maxlag:], network_prediction, \
                           multioutput='variance_weighted')
        
        r2List.append(r2Score)
    return r2List

#### Preparing Data

In [14]:
hub = 'Penn'
tune_hyp_params = False
pca_comps = 6

In [15]:
dataDir = '/home/mingyi/Dropbox/UrbanTemporalNetworks/processedData/'
file = dataDir + hub + 'VehicleByHour.csv'

In [16]:
# file = '/home/urwa/Documents/Projects/NYU Remote/project/data/JfkVehiceByHour.csv'

In [17]:
data = loadData(file)

Raw shape:  (2251320, 4)
Days:  365


In [18]:
data = getTimeSeries(data)

In [19]:
data.shape

(8760, 257)

In [20]:
data

DOLocationID     1    2    3    4    5    6    7    8    9    10   ...  254  \
Date       Hour                                                    ...        
2018-01-01 0       0    0    0    4    0    0    1    0    0    0  ...    0   
           1       0    0    0    8    0    0    6    0    0    0  ...    0   
           2       1    0    0    7    0    0    8    0    0    0  ...    0   
           3       5    0    0    3    0    0    6    0    0    0  ...    0   
           4       3    0    0    2    0    0    2    0    1    0  ...    0   
...              ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2018-12-31 19      2    0    1    6    0    0    1    0    0    0  ...    0   
           20      0    0    0   12    0    0    5    0    0    0  ...    0   
           21      0    0    0    8    0    0    3    0    0    0  ...    0   
           22      0    0    0    5    0    0    2    0    0    0  ...    0   
           23      0    0    0    1    0    0    3    0    0    1  ...    0   

DOLocationID     255  256  257  258  259  260  261  262  263  
Date       Hour                                               
2018-01-01 0       3    1    0    1    0    0    2    7    3  
           1      13    4    3    0    0    1    3    5   13  
           2       4    2    1    0    0    1    4    4   10  
           3       5    3    1    1    1    1    1    7    5  
           4       1    1    0    0    0    1    3    2    2  
...              ...  ...  ...  ...  ...  ...  ...  ...  ...  
2018-12-31 19      3   10    1    0    0    0    3    6    7  
           20      2    5    1    0    0    0    2    4    6  
           21      2    5    0    0    1    0    2    0    3  
           22      3    1    0    0    0    1    7    1    3  
           23      0    2    0    0    0    0    3    1    0  

[8760 rows x 257 columns]

In [21]:
matrix = data.values.astype(np.float64)

In [22]:
scaler, s_matrix = standardize(matrix)

In [23]:
pca,pcaData = getPCAFeatures(s_matrix,n=pca_comps)

In [24]:
pcaData.index = data.index
pcaData = pcaData.reset_index()

In [30]:
pcaData.to_csv('../../processedData/%spca%s.csv'%(hub.upper(),pca_comps),
               index=False)

In [25]:
externalDataDir = "/home/mingyi/Dropbox/UrbanTemporalNetworks/HongData/"
extFile = externalDataDir + hub.upper() + ".csv"

In [26]:
extDf = pd.read_csv(extFile)
print(extDf.shape)
extDf.head(2)

(8760, 46)


,date,arrival,fhv,yellow,vehicle,ifmon,iftue,ifwed,ifthu,iffri,...,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,18/1/1 0:00,0,147,319,466,1,0,0,0,0,...,19,7,13.0,-20.4,52,0,0.0,0.0,0,0
1,18/1/1 1:00,1,347,397,744,1,0,0,0,0,...,19,7,13.0,-20.4,52,0,0.0,0.0,0,0


In [27]:
extDf['date'] = pd.to_datetime(extDf['date'], yearfirst=True)
extDf.head(2)

,date,arrival,fhv,yellow,vehicle,ifmon,iftue,ifwed,ifthu,iffri,...,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,2018-01-01 00:00:00,0,147,319,466,1,0,0,0,0,...,19,7,13.0,-20.4,52,0,0.0,0.0,0,0
1,2018-01-01 01:00:00,1,347,397,744,1,0,0,0,0,...,19,7,13.0,-20.4,52,0,0.0,0.0,0,0


In [28]:
extDf['Hour'] = extDf['date'].dt.hour
extDf['Dow'] = extDf['date'].dt.dayofweek
extDf['Date'] = extDf['date'].dt.date

In [29]:
extDf.columns

Index(['date', 'arrival', 'fhv', 'yellow', 'vehicle', 'ifmon', 'iftue',
       'ifwed', 'ifthu', 'iffri', 'ifsat', 'ifsun', 'if0', 'if1', 'if2', 'if3',
       'if4', 'if5', 'if6', 'if7', 'if8', 'if9', 'if10', 'if11', 'if12',
       'if13', 'if14', 'if15', 'if16', 'if17', 'if18', 'if19', 'if20', 'if21',
       'if22', 'if23', 'maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow', 'Hour', 'Dow',
       'Date'],
      dtype='object')

In [30]:
selected_columns = ['Date', 'Hour', 'Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

In [31]:
extDf = extDf[selected_columns]

In [32]:
print(pcaData.shape)
print(extDf.shape)

(8760, 8)
(8760, 14)


In [33]:
pcaData['Date'] = pd.to_datetime(pcaData['Date'])
extDf['Date'] = pd.to_datetime(extDf['Date'])

In [34]:
pcaData = pd.merge(pcaData,extDf, on=['Date', 'Hour'], how='inner')
print(pcaData.shape)

(8760, 20)


In [35]:
pcaData.columns

Index(['Date', 'Hour', '1', '2', '3', '4', '5', '6', 'Dow', 'arrival',
       'maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd', 'cdd',
       'participation', 'newsnow', 'snowdepth', 'ifSnow'],
      dtype='object')

In [36]:
lagColumns = ['1', '2', '3', '4', '5', '6', 'arrival']
# lagColumns = ['1', '2', '3', 'arrival']

DateColumns = ['Date']

targetColumns = ['1', '2', '3', '4', '5', '6']

In [37]:
maxlag = 12

pcaData_lag = addLag(pcaData, maxlag, lagColumns)

pcaData_lag.shape

(8748, 104)

In [38]:
data.columns

Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            254, 255, 256, 257, 258, 259, 260, 261, 262, 263],
           dtype='int64', name='DOLocationID', length=257)

In [39]:
pcaData_lag

,Date,Hour,1,2,3,4,5,6,Dow,arrival,...,5_lag_11,6_lag_11,arrival_lag_11,1_lag_12,2_lag_12,3_lag_12,4_lag_12,5_lag_12,6_lag_12,arrival_lag_12
12,2018-01-01,12,-2.109717,-1.273276,-2.187085,1.342227,0.561549,-1.447407,0,2,...,-4.213746,0.785231,1.0,1.066354,3.055054,-0.439651,0.508365,1.075769,0.308136,0.0
13,2018-01-01,13,2.735838,-1.550082,-0.493573,-0.485399,0.758877,0.179975,0,4,...,0.160315,-4.490631,1.0,8.685871,3.502754,-1.378845,-1.533524,-4.213746,0.785231,1.0
14,2018-01-01,14,1.928464,-0.656030,-1.605285,-2.186646,-0.517429,-1.681551,0,4,...,0.265650,-1.177021,0.0,9.962452,5.593679,2.204155,-1.794186,0.160315,-4.490631,1.0
15,2018-01-01,15,4.399894,-1.245650,-1.849710,-1.786173,-0.097434,-0.610078,0,4,...,-0.073427,-1.749409,0.0,6.187023,7.039646,2.833590,0.087508,0.265650,-1.177021,0.0
16,2018-01-01,16,8.320408,0.462024,-1.214031,-2.953506,-0.809814,-1.704227,0,6,...,0.052865,-0.335715,0.0,0.782430,6.245589,2.065093,1.272655,-0.073427,-1.749409,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2018-12-31,19,5.025784,-1.653231,-1.846764,-3.459036,2.121074,0.596973,0,3,...,-0.662389,-2.100378,2.0,-7.139212,0.862787,0.359641,-1.075187,-0.396154,0.091690,1.0
8756,2018-12-31,20,5.382951,-1.941856,-1.257980,-2.150840,1.783696,0.584639,0,4,...,0.091009,-1.166719,4.0,-6.856731,0.545142,0.181022,0.778644,-0.662389,-2.100378,2.0
8757,2018-12-31,21,3.914864,-0.929810,0.987710,0.650458,3.866770,3.017943,0,3,...,-1.112528,-0.986341,3.0,-4.933395,1.023871,-0.581215,0.672369,0.091009,-1.166719,4.0
8758,2018-12-31,22,0.755087,0.862847,0.517217,1.005758,1.116948,-0.275292,0,2,...,-0.041566,-2.297913,4.0,-3.764957,-0.282021,-1.410743,0.793512,-1.112528,-0.986341,3.0


In [40]:
CommR2List = []
EdgeR2List = []
residualDf_list = []
rawList = []
networkPrediction = pd.DataFrame()

for m in range(1,13):
    print()

    print("month: ",m)
    month_index  = pd.to_datetime(pcaData_lag.Date).dt.month == m

    dataset_train = pcaData_lag[~month_index]
    dataset_test = pcaData_lag[month_index]
    print("Train Size: ",dataset_train.shape)
    print("Test Size: ",dataset_test.shape)


    X_train = dataset_train.drop(targetColumns+DateColumns , axis = 1)
    X_test = dataset_test.drop(targetColumns+DateColumns , axis = 1)
    y_train = dataset_train[targetColumns]
    y_test = dataset_test[targetColumns]



    rf2 = RandomForestRegressor(random_state = 2019, n_estimators=150, 
                               min_samples_split=3,
                               min_samples_leaf= 2, 
                               max_features= 'sqrt',
                               max_depth= None, 
                               bootstrap= False)

    rf2.fit(X_train,y_train)

    print("Train R2: ",rf2.score(X_train,y_train))
    test_r2 = rf2.score(X_test,y_test)
    print("Test R2: ",test_r2)


    pca_prediction = rf2.predict(X_test)

    residual = y_test - pca_prediction
    residual_df = dataset_test[['Date','Hour']]
    residual_df = pd.concat([residual_df,pd.DataFrame(residual)], axis =1)

    network_prediction = inverse_pca(pca_prediction,pca)
    network_prediction = inverse_standardize(network_prediction, scaler)
    
        # relu to convert all prediction to positive
#     network_prediction = np.log(1+np.e**network_prediction)
    # round up negative values to 0
#     network_prediction = np.where(network_prediction<0,0,network_prediction)
    
    network_prediction_df = pd.DataFrame(network_prediction)
    network_prediction_df.columns = data.columns
    networkPrediction = pd.concat([networkPrediction,network_prediction_df])
    edgeMonthIndex = [False] * maxlag + list(month_index)
    edge_r2 = r2_score(data[edgeMonthIndex], network_prediction, multioutput='variance_weighted')
    print("Edge R2: ",edge_r2)


    CommR2List.append(test_r2)
    EdgeR2List.append(edge_r2)
    residualDf_list.append(residual_df)
#     rawList.append()


month:  1
Train Size:  (8016, 104)
Test Size:  (732, 104)
Train R2:  0.9858007852416183
Test R2:  0.830033462895922
Edge R2:  0.6707309392624646

month:  2
Train Size:  (8076, 104)
Test Size:  (672, 104)
Train R2:  0.9862167359523013
Test R2:  0.7642548415483423
Edge R2:  0.700434608143839

month:  3
Train Size:  (8004, 104)
Test Size:  (744, 104)
Train R2:  0.9855525917227257
Test R2:  0.8571347930171511
Edge R2:  0.6841017224867082

month:  4
Train Size:  (8028, 104)
Test Size:  (720, 104)
Train R2:  0.985804007271607
Test R2:  0.8762059232474261
Edge R2:  0.739384736474159

month:  5
Train Size:  (8004, 104)
Test Size:  (744, 104)
Train R2:  0.9857974884457852
Test R2:  0.8759194300112324
Edge R2:  0.7418606540733098

month:  6
Train Size:  (8028, 104)
Test Size:  (720, 104)
Train R2:  0.98586634669251
Test R2:  0.8600397369088791
Edge R2:  0.722484022277267

month:  7
Train Size:  (8004, 104)
Test Size:  (744, 104)
Train R2:  0.9861441892093024
Test R2:  0.831165631194737
Edge R2:

In [59]:
networkPrediction['Date'] = data.reset_index().iloc[12:]['Date'].values
networkPrediction['Hour'] = data.reset_index().iloc[12:]['Hour'].values
networkPrediction.to_csv('/home/mingyi/Dropbox/UrbanTemporalNetworks/prediction/%sPCA%s.csv'%(hub,pca_comps),index=False)

In [60]:
print(np.mean(CommR2List))
print(np.mean(EdgeR2List))

0.8357098237615005
0.70522396823975


In [41]:
res_df = pd.concat(residualDf_list, axis = 0)
res_df.to_csv('../../Resid/'+hub+'PCA'+str(pca_comps)+'RFCV.csv',index=False)